In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
total_epoch=100
batch_size=100
learning_rate=0.0002
n_hidden=256
n_input=28*28
n_noise=128
X=tf.placeholder(tf.float32,[None,n_input])
Z=tf.placeholder(tf.float32,[None,n_noise])

In [6]:
#128 256 

G_W1=tf.Variable(tf.random_normal([n_noise,n_hidden],stddev=0.01))
G_b1=tf.Variable(tf.zeros([n_hidden]))
G_W2=tf.Variable(tf.random_normal([n_hidden,n_input],stddev=0.01))
G_b2=tf.Variable(tf.zeros([n_input]))

#256 784
D_W1=tf.Variable(tf.random_normal([n_input,n_hidden],stddev=0.01))
D_b1=tf.Variable(tf.zeros([n_hidden]))
D_W2=tf.Variable(tf.random_normal([n_hidden,1],stddev=0.01))
D_b2=tf.Variable(tf.zeros([1]))

In [7]:
#generator: 생성기
#discriminator:판별기
#cost funcrion :확률적함수를 사용
#지금까지는 MSE,Entropy:분류
#KL-Divergence:GAN,VAE-분포의 차를 확인하는 함수
#생성기->noise가 input데이터,cost function :KL-Divargence

def generator(noise_z): #128 128
    hidden=tf.nn.relu(tf.matmul(noise_z, G_W1)+G_b1)
    output=tf.nn.sigmoid(tf.matmul(hidden,G_W2)+G_b2)
    return output
def discriminator(inputs):
    hidden=tf.nn.relu(tf.matmul(inputs,D_W1)+D_b1)
    output=tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2)
    return output
def get_noise(batch_size,n_noise):
    return np.random.normal(size=(batch_size,n_noise))

In [11]:
G=generator(Z) #노이즈 =>이미지 생성
D_gene=discriminator(G) #이미지를 판별-분포를 확인한다
D_real=discriminator(X) #실제이미지의 분포를 확인

#log 확률값->정보량
#확률이 높아지면 정보량이 작아지고 
#확률이 낮아지면 정보량이 커짐 tfidf 의 논리를 고려할 것
#가장적합한 분포를 찾아냄 ->높은것 유리
loss_D=tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene))
loss_G=tf.reduce_mean(tf.log(D_gene)) #최대우도 추정법
D_var_list=[D_W1,D_b1,D_W2,D_b2]
G_var_list=[G_W1,G_b1,G_W2,G_b2]

#최대화하기 위해 -
train_D=tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,var_list=D_var_list)
train_G=tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list=G_var_list)
%matplotlib inline
sess=tf.Session()
sess.run(tf.global_variables_initializer())
total_batch=int(mnist.train.num_examples/batch_size)
loss_val_D,loss_val_G=0,0



In [15]:
#이미지 분포의 특징을 추출하는 망

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        noise=get_noise(batch_size,n_noise)
        #실제이미지 분포 
        _,loss_val_D=sess.run([train_D,loss_D],
                             feed_dict={X:batch_xs,Z:noise})
        #
        _,loss_val_G=sess.run([train_G,loss_G],
                             feed_dict={X:batch_xs,Z:noise})
    print('epoch:','%04d' % epoch,'D loss: {:.4}'.format(loss_val_D),'G loss: {:.4}'.format(loss_val_G))
    if epoch==0 or (epoch + 1) % 10==0:
        #노이즈로 부터 이미지를 생성:generator
        sample_size = 10
        noise=get_noise(sample_size,n_noise) #10 x 128 (noise)
        samples=sess.run(G,feed_dict={Z:noise})
        fig,ax=plt.subplots(1,sample_size,figsize=(sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off()#128=>784
            ax[i].imshow(np.reshape(samples[i],(28,28)))
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches='tight')
        plt.close(fig)
print('최적화완료')
        

epoch: 0000 D loss: -0.3128 G loss: -1.822
epoch: 0001 D loss: -0.2808 G loss: -2.362
epoch: 0002 D loss: -0.1796 G loss: -3.172
epoch: 0003 D loss: -0.2308 G loss: -2.721
epoch: 0004 D loss: -0.1792 G loss: -2.92
epoch: 0005 D loss: -0.292 G loss: -2.953
epoch: 0006 D loss: -0.1701 G loss: -3.389
epoch: 0007 D loss: -0.3653 G loss: -2.375
epoch: 0008 D loss: -0.2716 G loss: -2.706
epoch: 0009 D loss: -0.3521 G loss: -2.675
epoch: 0010 D loss: -0.3837 G loss: -2.668
epoch: 0011 D loss: -0.3954 G loss: -2.603
epoch: 0012 D loss: -0.3327 G loss: -2.31
epoch: 0013 D loss: -0.3874 G loss: -2.52
epoch: 0014 D loss: -0.273 G loss: -2.689
epoch: 0015 D loss: -0.3179 G loss: -2.666
epoch: 0016 D loss: -0.3788 G loss: -2.527
epoch: 0017 D loss: -0.4643 G loss: -2.264
epoch: 0018 D loss: -0.5703 G loss: -2.306
epoch: 0019 D loss: -0.4706 G loss: -2.386
epoch: 0020 D loss: -0.5245 G loss: -2.137
epoch: 0021 D loss: -0.352 G loss: -2.685
epoch: 0022 D loss: -0.4562 G loss: -2.31
epoch: 0023 D loss